In [108]:
import pandas as pd
df1 = pd.read_csv('Data/xerces-1.2.csv')
df2 = pd.read_csv('Data/tomcat.csv')
df = pd.concat([df1, df2])

In [109]:
df = df.drop(['name', 'version', 'name.1'], axis=1)

In [110]:
import numpy as np
X_source = np.array(df.drop(['bug'], axis=1), dtype='float32')
y_source = np.array(df['bug'], dtype='int8')
y_source[y_source>0] = 1

In [111]:
df_t = pd.read_csv('Data/arc.csv')
X_target = np.array(df_t.drop(['name', 'version', 'name.1', 'bug'], axis=1), dtype='float32')
y_target = np.array(df_t['bug'], dtype='int8')
y_target[y_target>0] = 1


In [112]:
def similarityWeight(X_sample, X_target):
    max_elem = np.amax(X_target, axis=0)
    min_elem = np.amin(X_target, axis=0)
    
    similarityValue = []
    
    s_max = 0
    
    for i in range(X_sample.shape[0]):
        s = 0
        for j in range(X_sample.shape[1]):
            if min_elem[j] <= X_sample[i,j] <= max_elem[j]:
                s += 1
        
        s = s/X_sample.shape[1]
        s_max = max(s_max, s)
        similarityValue.append(s)
    
    similarityWeights = np.array(similarityValue)/s_max
    
    return similarityWeights

In [113]:
def dataSelector(X_sample, y_sample, similarityWeights, threshold):
    X = []
    y = []
    for i in range(len(y_sample)):
        if similarityWeights[i] >= threshold:
            X.append(X_sample[i])
            y.append(y_sample[i])
    X = np.asarray(X)
    y = np.asarray(y)
    return X, y

In [114]:
threshold = 1
sw = similarityWeight(X_source, X_target)
X, y = dataSelector(X_source, y_source, sw, threshold)
len(y)

979

In [115]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)

In [116]:
from imblearn.over_sampling import ADASYN
imb = ADASYN()
# X_train, y_train = imb.fit_resample(X_train, y_train)

In [117]:
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score, matthews_corrcoef, make_scorer
from sklearn.naive_bayes import GaussianNB

from sklearn.metrics import f1_score, make_scorer

f1 = make_scorer(f1_score)

param_grid = {  
 'n_estimators': [5, 10, 15, 50],
 'max_samples' : [0.6, 0.8, 1.0],
}

clf = GridSearchCV(BaggingClassifier(base_estimator=AdaBoostClassifier()), param_grid=param_grid, cv=5)

# params_NB = {'var_smoothing': np.logspace(0,-9, num=300)}
# clf = RandomizedSearchCV(estimator=GaussianNB(), 
#                  param_distributions=params_NB, 
#                  scoring=make_scorer(matthews_corrcoef),
#                  cv=5)

clf.fit(X_train, y_train)


GridSearchCV(cv=5,
             estimator=BaggingClassifier(base_estimator=AdaBoostClassifier()),
             param_grid={'max_samples': [0.6, 0.8, 1.0],
                         'n_estimators': [5, 10, 15, 50]})

In [118]:
y_train_pred = clf.predict(X_train)
print("F1 Score: ", f1_score(y_train, y_train_pred))
print("MCC score: ", matthews_corrcoef(y_train, y_train_pred))

F1 Score:  0.15384615384615385
MCC score:  0.2767546517312798


In [119]:
y_test_pred = clf.predict(X_test)
print("F1 Score: ", f1_score(y_test, y_test_pred))
print("MCC score: ", matthews_corrcoef(y_test, y_test_pred))

F1 Score:  0.0
MCC score:  -0.03485917507547887


In [120]:
y_target_pred = clf.predict(X_target)
print("F1 Score: ", f1_score(y_target, y_target_pred))
print("MCC score: ", matthews_corrcoef(y_target, y_target_pred))

F1 Score:  0.0
MCC score:  0.0
